In [2]:
# In[1]:
# --- 1. Import Necessary Libraries ---
# These are the libraries needed to make web requests, parse URLs, and handle data.

import requests
import json
import time
from urllib.parse import urlparse

# In[2]:
# --- 2. Define Helper Functions for Scraping ---
# We'll break down the logic from your Django view into reusable functions.

# This function is specifically for scraping a single Reddit post and its comments.
def scrape_reddit_post(api_url, headers):
    """Scrapes a single Reddit post and its top-level comments."""
    print(f"Fetching post data from: {api_url}")
    
    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()  # Raises an error for bad responses (4xx or 5xx)
        data = response.json()

        # --- Extract Post Data ---
        post_data = data[0]['data']['children'][0]['data']
        post_title = post_data.get('title', 'No Title')
        post_author = post_data.get('author', 'Unknown Author')
        post_text = post_data.get('selftext', 'No content text.')
        
        content_lines = [
            f"Title: {post_title}",
            f"Author: u/{post_author}",
            "--- POST CONTENT ---",
            post_text,
            "\n--- COMMENTS ---"
        ]

        # --- Extract Comments Data ---
        comments_data = data[1]['data']['children']
        for comment in comments_data:
            if 'data' in comment and 'body' in comment['data']:
                comment_author = comment['data'].get('author', 'Unknown')
                comment_body = comment['data'].get('body', '')
                # Skip '[removed]' or '[deleted]' comments
                if comment_body not in ['[removed]', '[deleted]']:
                    content_lines.append(f"\n> u/{comment_author}:\n{comment_body}\n")
        
        return "\n".join(content_lines)

    except requests.RequestException as e:
        return f"Error: Failed to retrieve data from Reddit API. Details: {e}"
    except (KeyError, IndexError, json.JSONDecodeError) as e:
        return f"Error: Failed to parse the Reddit API response. Details: {e}"

# This function scrapes all the posts on the front page of a subreddit.
def scrape_subreddit(api_url, headers):
    """Scrapes the top posts and their comments from a subreddit's main page."""
    print(f"Fetching subreddit data from: {api_url}")
    
    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()
        data = response.json()

        subreddit_name = data['data']['children'][0]['data']['subreddit']
        posts = data['data']['children']
        
        content_lines = [f"Scraped Posts from r/{subreddit_name}:\n" + "="*40]

        for i, post_item in enumerate(posts):
            post_data = post_item['data']
            post_title = post_data.get('title', 'No Title')
            post_author = post_data.get('author', 'Unknown')
            permalink = post_data.get('permalink')

            if not permalink:
                continue

            content_lines.append(f"\n\n--- POST {i+1}: {post_title} (by u/{post_author}) ---")
            
            # Construct the API URL for the individual post
            post_api_url = f"https://www.reddit.com{permalink.rstrip('/')}.json"
            
            # Reuse the post scraper function for each post in the subreddit
            post_content = scrape_reddit_post(post_api_url, headers)
            content_lines.append(post_content)
            
            # Be respectful to Reddit's API and add a small delay
            time.sleep(0.5)

        return "\n".join(content_lines)

    except requests.RequestException as e:
        return f"Error: Failed to retrieve subreddit data. Details: {e}"
    except (KeyError, IndexError, json.JSONDecodeError) as e:
        return f"Error: Failed to parse the subreddit API response. Details: {e}"

# In[3]:
# --- 3. Main Controller Function ---
# This function decides whether the URL is a post or a subreddit and calls the correct helper.

def scrape_reddit(url):
    """
    Main function to handle Reddit scraping.
    It checks the URL type and calls the appropriate scraping function.
    """
    if not url:
        return "Error: Please provide a URL."

    parsed_url = urlparse(url)
    
    # All Reddit API requests need a custom User-Agent
    headers = {'User-Agent': 'My-Local-Scraper/1.0'}
    
    # Convert the regular Reddit URL to its JSON API equivalent
    api_url = url.rstrip('/') + ".json"

    # LOGIC 1: Handle a specific post URL (contains /comments/)
    if '/comments/' in parsed_url.path:
        return scrape_reddit_post(api_url, headers)
        
    # LOGIC 2: Handle a subreddit URL (e.g., /r/Python/)
    elif parsed_url.path.startswith('/r/'):
        return scrape_subreddit(api_url, headers)
        
    # LOGIC 3: Invalid Reddit URL
    else:
        return "Error: This appears to be a Reddit URL, but not a valid post or subreddit page."


# In[4]:
# --- 4. Execution Block ---
# This is where you run the script.
# Just change the 'target_url' to the Reddit page you want to scrape.

if __name__ == "__main__":
    
    # --- CHOOSE A URL TO SCRAPE ---
    # Example 1: A single post
    # target_url = "https://www.reddit.com/r/Python/comments/10c3jix/python_tricks_that_you_should_know/"
    
    # Example 2: A subreddit's main page
    target_url = "https://www.reddit.com/r/ChevroletSilverado"

    print(f"--- Starting scrape for: {target_url} ---\n")
    
    # Call the main function and get the content
    scraped_data = scrape_reddit(target_url)
    
    # Print the results
    print("\n--- SCRAPING COMPLETE ---")
    print(scraped_data)

    # Optional: Save the output to a file
    with open("reddit_scrape.txt", "w", encoding="utf-8") as f:
        f.write(scraped_data)
    print("\n--- Data saved to reddit_scrape.txt ---")

--- Starting scrape for: https://www.reddit.com/r/ChevroletSilverado ---

Fetching subreddit data from: https://www.reddit.com/r/ChevroletSilverado.json
Fetching post data from: https://www.reddit.com/r/ChevroletSilverado/comments/1mz8y8s/help_hard_wiring_a_dash_cam.json
Fetching post data from: https://www.reddit.com/r/ChevroletSilverado/comments/1l2cxcu/rhino_liner.json
Fetching post data from: https://www.reddit.com/r/ChevroletSilverado/comments/1kw0lo4/2019_silverado_shake.json
Fetching post data from: https://www.reddit.com/r/ChevroletSilverado/comments/1ijlaqp/auto_stop.json
Fetching post data from: https://www.reddit.com/r/ChevroletSilverado/comments/1idmdod/can_i_get_some_love.json
Fetching post data from: https://www.reddit.com/r/ChevroletSilverado/comments/1huitby/spare_tire.json
Fetching post data from: https://www.reddit.com/r/ChevroletSilverado/comments/1hpigsk/help.json
Fetching post data from: https://www.reddit.com/r/ChevroletSilverado/comments/1hpfquj/would_you_recomme